# Sesión 1 - Introducción a Sagemaker 

## Inicio de sesión

In [1]:
import sagemaker

try:
    role = sagemaker.get_execution_role()
except:
    role = "arn:aws:iam::827345860551:role/SageMakerExecutionRole"

sesion = sagemaker.Session()
bucket = sesion.default_bucket()
region = sesion.boto_session.region_name

## Subida de datos

In [3]:
import boto3
import pandas as pd
import numpy as np
import sagemaker
from sklearn.datasets import load_iris

data = load_iris()
df = pd.DataFrame(
    data=np.c_[data['data'], data['target']],
    columns= data['feature_names'] + ['Species']
    )

df.to_csv("iris.csv",index=False)

s3_path = sesion.upload_data(
    path="iris.csv",
    bucket=bucket,
    key_prefix="curso_sagemaker/data"
)

print(s3_path)

s3://sagemaker-eu-west-1-827345860551/curso_sagemaker/data/iris.csv


## Creación del Estimator

In [4]:
from sagemaker.sklearn.estimator import SKLearn

sklearn = SKLearn(
    source_dir='./src',
    entry_point='train.py',
    framework_version='0.23-1',
    instance_type="ml.c4.xlarge",
    role=role,
    sagemaker_session=sesion,
    hyperparameters={
        "min_leaf_nodes": 3,
        "n_estimators": 10,
        "target": "Species"
    }
)

## Entrenamiento del modelo

In [5]:
sklearn.fit(inputs={"train": s3_path})

2022-03-30 07:52:49 Starting - Starting the training job...
2022-03-30 07:52:51 Starting - Launching requested ML instancesProfilerReport-1648626768: InProgress
......
2022-03-30 07:54:18 Starting - Preparing the instances for training......
2022-03-30 07:55:23 Downloading - Downloading input data...
2022-03-30 07:55:44 Training - Downloading the training image...
2022-03-30 07:56:24 Uploading - Uploading generated training model2022-03-30 07:56:10,808 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2022-03-30 07:56:10,810 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-03-30 07:56:10,819 sagemaker_sklearn_container.training INFO     Invoking user training script.
2022-03-30 07:56:11,209 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/miniconda3/bin/python -m pip install -r requirements.txt
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 66.6 MB/s eta 0:00:00
  Att

## Despliegue del modelo

In [2]:
predictor = sklearn.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.xlarge"
)

## Predicción de datos

In [6]:
from sklearn.model_selection import train_test_split

X = df.loc[:, df.columns != "Species"].values
y = df.loc[:, df.columns == "Species"].values.ravel()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=28)

response = predictor.predict(X_test)
print(response)

[0. 1. 1. 0. 2. 1. 2. 1. 1. 0. 2. 0. 1. 1. 2. 0. 2. 2. 2. 1. 0. 0. 1. 2.
 1. 0. 2. 2. 0. 2. 0. 2. 1. 0. 2. 1. 2. 1.]


## Borrado del endpoint

In [7]:
predictor.delete_endpoint()